In [2]:
import requests
import pandas as pd
import time
import numpy as np

In [7]:
API_KEY = "33c3009d4610d7c8a3484f3bc865055b"          
BASE_URL = "https://api.themoviedb.org/3"

# The 19 IDs given in the lab (0 is invalid → we remove it)
movie_ids = [
    0, 299534, 19995, 140607, 299536, 597, 135397, 420818,
    24428, 168259, 99861, 284054, 12445, 181808, 330457,
    351286, 109445, 321612, 260513
    # 0 was removed because TMDb has no movie with ID 0 → it returns 404
]

# === 2. Function to get one movie (with credits appended) ===
def fetch_movie(movie_id: int) -> dict:
    url = f"{BASE_URL}/movie/{movie_id}"
    params = {
        "api_key": API_KEY,
        "language": "en-US",
        "append_to_response": "credits"   # gets cast + crew in the same call
    }
    
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        return response.json()
    elif response.status_code == 404:
        print(f"Movie ID {movie_id} not found (404)")
        return None
    else:
        print(f"Error {response.status_code} for ID {movie_id}: {response.text}")
        return None
    
 

In [8]:

# === 3. Fetch all movies with polite delay (TMDb allows ~50 requests/sec, but we play nice) ===
print("Starting to fetch movies...")
raw_movies_data = []

for idx, mid in enumerate(movie_ids, 1):
    print(f"  [{idx:02d}/{len(movie_ids)}] Fetching ID {mid}...", end=" ")
    movie = fetch_movie(mid)
    if movie:
        raw_movies_data.append(movie)
        print("Success")
    else:
        print("Failed")
    
    time.sleep(0.22)  # ~4–5 requests per second → stays far below rate limit

print(f"\nFinished! Successfully fetched {len(raw_movies_data)} movies.")


Starting to fetch movies...
  [01/19] Fetching ID 0... Movie ID 0 not found (404)
Failed
  [02/19] Fetching ID 299534... Success
  [03/19] Fetching ID 19995... Success
  [04/19] Fetching ID 140607... Success
  [05/19] Fetching ID 299536... Success
  [06/19] Fetching ID 597... Success
  [07/19] Fetching ID 135397... Success
  [08/19] Fetching ID 420818... Success
  [09/19] Fetching ID 24428... Success
  [10/19] Fetching ID 168259... Success
  [11/19] Fetching ID 99861... Success
  [12/19] Fetching ID 284054... Success
  [13/19] Fetching ID 12445... Success
  [14/19] Fetching ID 181808... Success
  [15/19] Fetching ID 330457... Success
  [16/19] Fetching ID 351286... Success
  [17/19] Fetching ID 109445... Success
  [18/19] Fetching ID 321612... Success
  [19/19] Fetching ID 260513... Success

Finished! Successfully fetched 18 movies.


In [10]:
# === 4. Convert the list of JSONs into a Pandas DataFrame ===
df_raw = pd.DataFrame(raw_movies_data)


print(f"\nDataFrame shape: {df_raw.shape}")
print("Columns:", list(df_raw.columns))
print("\nFirst 2 rows sample:")
display(df_raw[['id', 'title', 'release_date', 'budget', 'revenue']].head(10))


df_raw.to_json("raw_tmdb_movies.json", orient="records", indent=2)

print("\nRaw data saved as raw_tmdb_movies.json and .csv")


DataFrame shape: (18, 27)
Columns: ['adult', 'backdrop_path', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'origin_country', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count', 'credits']

First 2 rows sample:


,id,title,release_date,budget,revenue
0,299534,Avengers: Endgame,2019-04-24,356000000,2799439100
1,19995,Avatar,2009-12-15,237000000,2923706026
2,140607,Star Wars: The Force Awakens,2015-12-15,245000000,2068223624
3,299536,Avengers: Infinity War,2018-04-25,300000000,2052415039
4,597,Titanic,1997-11-18,200000000,2264162353
5,135397,Jurassic World,2015-06-06,150000000,1671537444
6,420818,The Lion King,2019-07-12,260000000,1662020819
7,24428,The Avengers,2012-04-25,220000000,1518815515
8,168259,Furious 7,2015-04-01,190000000,1515400000
9,99861,Avengers: Age of Ultron,2015-04-22,235000000,1405403694



Raw data saved as raw_tmdb_movies.json and .csv


In [11]:
df_raw.info()
print(f"Original shape: {df_raw.shape}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  18 non-null     bool   
 1   backdrop_path          18 non-null     object 
 2   belongs_to_collection  16 non-null     object 
 3   budget                 18 non-null     int64  
 4   genres                 18 non-null     object 
 5   homepage               18 non-null     object 
 6   id                     18 non-null     int64  
 7   imdb_id                18 non-null     object 
 8   origin_country         18 non-null     object 
 9   original_language      18 non-null     object 
 10  original_title         18 non-null     object 
 11  overview               18 non-null     object 
 12  popularity             18 non-null     float64
 13  poster_path            18 non-null     object 
 14  production_companies   18 non-null     object 
 15  producti

##### STEP 2 – DATA CLEANING(PHASE 1)

In [32]:
df_clean = df_raw.copy()
print(f"Starting with shape: {df_clean.shape}")

Starting with shape: (18, 27)


In [33]:
# === 1. Drop irrelevant columns ===
drop_cols = ['adult', 'imdb_id', 'original_title', 'video', 'homepage']
df_clean = df_clean.drop(columns=[col for col in drop_cols if col in df_clean.columns])
print(f"After dropping columns: {df_clean.shape}")


# === 2. Helper functions ===
def extract_collection_name(collection):
    if isinstance(collection, dict) and 'name' in collection:
        return collection['name']
    return np.nan

def extract_genres(genres_list):
    if isinstance(genres_list, list) and len(genres_list) > 0:
        return '|'.join([genre['name'] for genre in genres_list if 'name' in genre])
    return np.nan

def extract_spoken_languages(languages_list):
    if isinstance(languages_list, list) and len(languages_list) > 0:
        return '|'.join([lang['name'] for lang in languages_list if 'name' in lang])
    return np.nan

def extract_production_countries(countries_list):
    if isinstance(countries_list, list) and len(countries_list) > 0:
        return '|'.join([country['name'] for country in countries_list if 'name' in country])
    return np.nan

def extract_production_companies(companies_list):
    if isinstance(companies_list, list) and len(companies_list) > 0:
        return '|'.join([company['name'] for company in companies_list if 'name' in company])
    return np.nan

def extract_cast(credits):
    """Extract top cast members as 'Actor1|Actor2|...' or NaN."""
    if isinstance(credits, dict) and 'cast' in credits:
        cast_list = credits['cast'][:5]  # Top 5 actors
        return '|'.join([actor['name'] for actor in cast_list if 'name' in actor])
    return np.nan

def extract_cast_size(credits):
    if isinstance(credits, dict) and 'cast' in credits:
        return len(credits['cast'])
    return np.nan

def extract_crew_size(credits):
    if isinstance(credits, dict) and 'crew' in credits:
        return len(credits['crew'])
    return np.nan

def extract_director(credits):
    if isinstance(credits, dict) and 'crew' in credits:
        for person in credits['crew']:
            if person.get('job') == 'Director':
                return person.get('name', 'Unknown')
    return 'Unknown'

After dropping columns: (18, 22)


In [34]:
# === 3. Extract to TEMPORARY columns (different names) ===
print("\nExtracting nested fields...")

df_clean['collection_name'] = df_clean['belongs_to_collection'].apply(extract_collection_name)
df_clean['genre_names'] = df_clean['genres'].apply(extract_genres)
df_clean['language_codes'] = df_clean['spoken_languages'].apply(extract_spoken_languages)
df_clean['country_names'] = df_clean['production_countries'].apply(extract_production_countries)
df_clean['company_names'] = df_clean['production_companies'].apply(extract_production_companies)

# Extract from credits BEFORE dropping it
df_clean['cast'] = df_clean['credits'].apply(extract_cast)
df_clean['cast_size'] = df_clean['credits'].apply(extract_cast_size)
df_clean['crew_size'] = df_clean['credits'].apply(extract_crew_size)
df_clean['director'] = df_clean['credits'].apply(extract_director)

# === 4. NOW drop the raw JSON columns ===
json_cols = ['belongs_to_collection', 'genres', 'production_countries', 
             'production_companies', 'spoken_languages', 'credits']
df_clean = df_clean.drop(columns=[col for col in json_cols if col in df_clean.columns])

# === 5. Rename extracted columns to match required names ===
df_clean = df_clean.rename(columns={
    'genre_names': 'genres',
    'language_codes': 'spoken_languages',
    'country_names': 'production_countries',
    'company_names': 'production_companies'
})


Extracting nested fields...


### STEP 2 (PHASE 2) HANDLING MISSING DATA 

In [ ]:
# === 6. Inspect extracted columns ===
print("\n=== INSPECTION: Value Counts ===")
inspection_cols = ['collection_name', 'genres', 'spoken_languages', 
                   'production_countries', 'production_companies']

for col in inspection_cols:
    if col in df_clean.columns:
        print(f"\n{col.upper()}:")
        print(df_clean[col].value_counts().head(5))
        print(f"  → Unique: {df_clean[col].nunique()}, NaNs: {df_clean[col].isna().sum()}")



=== INSPECTION: Value Counts ===

COLLECTION_NAME:
collection_name
The Avengers Collection     4
Star Wars Collection        2
Frozen Collection           2
Jurassic Park Collection    2
Avatar Collection           1
Name: count, dtype: int64
  → Unique: 10, NaNs: 2

GENRES:
genres
Adventure|Action|Science Fiction             3
Action|Adventure|Science Fiction|Thriller    2
Action|Adventure|Science Fiction             2
Action|Adventure|Fantasy|Science Fiction     1
Drama|Romance                                1
Name: count, dtype: int64
  → Unique: 14, NaNs: 0

SPOKEN_LANGUAGES:
spoken_languages
English                                              9
English|日本語|                                         1
English|Español                                      1
English|                                             1
English|Français|Deutsch|svenska|Italiano|Pусский    1
Name: count, dtype: int64
  → Unique: 10, NaNs: 0

PRODUCTION_COUNTRIES:
production_countries
United States of America  

In [26]:
print("\n=== Sample of cleaned data ===")
df_clean.head(3)



=== Sample of cleaned data ===


,backdrop_path,budget,id,origin_country,original_language,overview,popularity,poster_path,release_date,revenue,...,vote_count,collection_name,genres,spoken_languages,production_countries,production_companies,cast,cast_size,crew_size,director
0,/9wXPKruA6bWYk2co5ix6fH59Qr8.jpg,356000000,299534,[US],en,After the devastating events of Avengers: Infi...,14.7068,/bR8ISy1O9XQxqiy0fQFw2BX72RQ.jpg,2019-04-24,2799439100,...,26983,The Avengers Collection,Adventure|Science Fiction|Action,English|日本語|,United States of America,Marvel Studios,Robert Downey Jr.|Chris Evans|Mark Ruffalo|Chr...,105,607,Joe Russo
1,/7JNzw1tSZZEgsBw6lu0VfO2X2Ef.jpg,237000000,19995,[US],en,"In the 22nd century, a paraplegic Marine is di...",39.5744,/gKY6q7SjCkAU6FqvqWybDYgUKIF.jpg,2009-12-15,2923706026,...,32887,Avatar Collection,Action|Adventure|Fantasy|Science Fiction,English|Español,United States of America|United Kingdom,Dune Entertainment|Lightstorm Entertainment|20...,Sam Worthington|Zoe Saldaña|Sigourney Weaver|S...,65,990,James Cameron
2,/8BTsTfln4jlQrLXUBquXJ0ASQy9.jpg,245000000,140607,[US],en,Thirty years after defeating the Galactic Empi...,8.6150,/wqnLdwVXoBjKibFRR5U3y0aDUhs.jpg,2015-12-15,2068223624,...,20107,Star Wars Collection,Adventure|Action|Science Fiction,English,United States of America,Lucasfilm Ltd.|Bad Robot,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,183,262,J.J. Abrams


In [35]:
# 7. Drop duplicates + critical missing
df_clean = df_clean.drop_duplicates(subset='id')
df_clean = df_clean.dropna(subset=['id', 'title'])

# 8. Keep only rows with meaningful data (at least 10 non-null columns)
df_clean = df_clean.dropna(thresh=10)

# 9. Keep only Released movies
if 'status' in df_clean.columns:
    df_clean = df_clean[df_clean['status'] == 'Released']
    df_clean = df_clean.drop(columns='status')

# 10. FINAL COLUMN ORDER
final_column_order = [
    'id', 'title', 'tagline', 'release_date', 'genres', 'collection_name',
    'original_language', 'budget_musd', 'revenue_musd',
    'production_companies', 'production_countries',
    'vote_count', 'vote_average', 'popularity', 'runtime',
    'overview', 'spoken_languages', 'poster_path',
    'cast', 'cast_size', 'director', 'crew_size'
]


# Reindex to match required order (only existing columns)
existing_cols = [col for col in final_column_order if col in df_clean.columns]
df_final = df_clean[existing_cols].copy()

# 11. Reset index
df_final = df_final.reset_index(drop=True)
print(f"\nFinal cleaned DataFrame shape: {df_final.shape}")


Final cleaned DataFrame shape: (18, 20)


### Step 3: KPI Implementation & Analysis
